<a href="https://colab.research.google.com/github/gauravgund/gauravgund/blob/master/Exkaldi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#rshi
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:

!7za x /content/gdrive/MyDrive/kaldi_docs/kaldi.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/MyDrive/kaldi_docs/                                             1 file, 7146613415 bytes (6816 MiB)

Extracting archive: /content/gdrive/MyDrive/kaldi_docs/kaldi.zip
  3% 4096 Open              --
Path = /content/gdrive/MyDrive/kaldi_docs/kaldi.zip
Type = zip
Physical Size = 7146613415
64-bit = +

  0%      0% 563 - content/kaldi/egs/swahili/s5/steps/conf/train_calibration.sh                                                                         0% 1392 - content/ka

In [4]:
! pip install https://github.com/kpu/kenlm/archive/master.zip
! pip install exkaldi

     |████████████████████████████████| 542kB 4.3MB/s 
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2334921 sha256=91d468944e7019d390b932b78b3bc0a11363db1daf6e6416207efbc662422275
  Stored in directory: /tmp/pip-ephem-wheel-cache-0psi6l7t/wheels/2d/32/73/e3093c9d11dc8abf79c156a4db1a1c5631428059d4f9ff2cba
Successfully built kenlm
     |████████████████████████████████| 1.2MB 7.8MB/s 


In [41]:
import os
os.environ["PATH"] += os.pathsep + '/content/content/kaldi'

In [6]:
!pip show exkaldi


Name: exkaldi
Version: 1.3.5.4
Summary: ExKaldi Automatic Speech Recognition Toolkit
Home-page: https://github.com/wangyu09/exkaldi
Author: Wang Yu
Author-email: wangyu@alps-lab.org
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy
Required-by: 


In [43]:
import os
import exkaldi
exkaldi.info.reset_kaldi_root('/content/content/kaldi')

In [44]:
! python3 -c "import exkaldi"

exkaldi.info.reset_kaldi_root( yourPath )
If not, ERROR will occur when implementing some core functions.


In [10]:
exkaldi.info.reset_kaldi_root

<bound method ExKaldiInfo.reset_kaldi_root of ExKaldiInfo(version='1.3.5.4', major='1', minor='3', patch='5', upload='4')>

In [11]:
import exkaldi
import os

In [12]:
ExKaldiInfo = exkaldi.info

ExKaldiInfo

ExKaldiInfo(version='1.3.5.4', major='1', minor='3', patch='5', upload='4')

In [13]:
ExKaldiInfo.KALDI       

Kaldi(version='5.5', major='5', minor='5')

In [17]:
%cd /content/
! mkdir kaldi_exp

/content


In [18]:
%cp /content/gdrive/MyDrive/kaldi_docs/wav.scp /content/kaldi_exp/wap.scp
%cp /content/gdrive/MyDrive/kaldi_docs/spk2utt.txt /content/kaldi_exp/spk2utt.txt
%cp /content/gdrive/MyDrive/kaldi_docs/utt2spk.txt /content/kaldi_exp/utt2spk.txt

In [30]:
%cp /content/gdrive/MyDrive/kaldi_docs/lexicon.txt /content/kaldi_exp/lexicon.txt

In [21]:
# rename file
# spk2utt.txt to spk2utt
# utt2spk.txt to utt2spk

In [19]:
scpFile = '/content/kaldi_exp/wap.scp'
feat = exkaldi.compute_mfcc(scpFile, name="mfcc")

In [22]:
feat.dim

13

In [23]:
spk2uttFile = '/content/kaldi_exp/spk2utt'
cmvn = exkaldi.compute_cmvn_stats(feat, spk2utt=spk2uttFile, name="cmvn")

cmvn

In [25]:
utt2spkFile = '/content/kaldi_exp/utt2spk'

feat = exkaldi.use_cmvn(feat, cmvn, utt2spk=utt2spkFile)



KaldiProcessError: ignored

In [26]:
feat.name

'mfcc'

In [27]:
%cd /content/kaldi_exp
!mkdir exp/
!touch exp/train_mfcc_cmvn.ark 

/content/kaldi_exp


In [28]:

featFile = '/content/kaldi_exp/exp/train_mfcc_cmvn.ark'
exkaldi.utils.make_dependent_dirs(path=featFile, pathIsFile=True)

featIndex = feat.save(featFile, returnIndexTable=True)

In [29]:
feat = featIndex.fetch(arkType="feat")

In [31]:
lexiconFile = '/content/kaldi_exp/lexicon.txt'

silWords={"<SIL>":"<SIL>",  # silence and its' pronunciation
          "<SPN>":"<SPN>"}  # spoken noise and its' pronunciation
unkSymbol={"<UNK>":"<SPN>"}  # unknown symbol and its' pronunciation
optionalSilPhone = "<SIL>"  # optional silence

lexicons = exkaldi.decode.graph.lexicon_bank(lexiconFile,
                                             silWords,
                                             unkSymbol, 
                                             optionalSilPhone, 
                                             positionDependent = False,
                                             shareSilPdf = False )

lexicons

In [32]:
lexicons.view

['lexiconp',
 'disambig',
 'lexiconp_disambig',
 'silence_phones',
 'optional_silence',
 'nonsilence_phones',
 'phone_map',
 'silence_phone_map',
 'nonsilence_phone_map',
 'extra_questions',
 'silence',
 'nonsilence',
 'context_indep',
 'wdisambig',
 'wdisambig_phones',
 'wdisambig_words',
 'align_lexicon',
 'oov',
 'sets',
 'roots',
 'phones',
 'words']

In [33]:
%cd /content/kaldi_exp

!touch exp/words.txt

/content/kaldi_exp


In [34]:
outFile = '/content/kaldi_exp/exp/words.txt'

exkaldi.utils.make_dependent_dirs(path=outFile, pathIsFile=True)

lexicons.dump_dict(name="words", fileName=outFile, dumpInt=False)

'/content/kaldi_exp/exp/words.txt'

In [35]:
%cd /content/kaldi_exp

!touch exp/L.fst
!touch exp/L_disambig.fst
! touch exp/lexicons.lex

/content/kaldi_exp


In [36]:
Lfile = '/content/kaldi_exp/exp/L.fst'
exkaldi.decode.graph.make_L(lexicons, outFile=Lfile, useDisambigLexicon=False)

fstarcsort: error while loading shared libraries: libfstscript.so.10: cannot open shared object file: No such file or directory
fstcompile: error while loading shared libraries: libfstscript.so.10: cannot open shared object file: No such file or directory



KaldiProcessError: ignored